<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.6: Testers2
**Prev | 上一个小节： [Control Flow | 控制流](2.3_control_flow.ipynb)**<br>
**Next | 下一个小节： [Generators: Parameters | 生成器: 变量](3.1_parameters.ipynb)**

## Motivation | 动机
The Chisel team has been working on an improved testing framework. Imaginatively titled "testers2", it provides the following improvements .

- suitable for both unit tests and system integration tests
- designed for composable abstractions and layering
- highly usable, encouraging unit tests by making it as easy, painless (avoiding boilerplate and other nonsense), and useful as possible to write them

Chisel 团队已经在着手优化测试架构了。名字暂定为 "testers2"，它提供了如下的提升：

+ 适合单元测试和系统集成测试
+ 被设计成可组合的抽象和分层
+ 非常有用，它被设计为简单、无痛苦（避免死板和其他无意义的内容）、易写的测试，使得单元测试更加简单。

### Planned | 计划
- ablity to target multiple backends and simulators (possibly requiring a link to Scala, if the testvector is not static, or using a limited test constructing API subset, when synthesizing to FPGA)
- will be included in base chisel3, to avoid packaging and dependency nightmares


+ 能够使用多种后端和仿真器（可能需要与 Scala 联系在一起，如果测试向量不是静态的话，或者需要综合到 FPGA 上时，可能需要使用一些特定的测试子集端口）
+ 能够被包含在 chisel3 中，从而避免包以及依赖的问题


## Setup | 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.experimental.BundleLiterals._
import chisel3.tester._
import chisel3.tester.RawTester.test

>This bootcamp requires some slight differences from the imports you might see 
elsewhere for chisel. The `import chisel3.tester.RawTester.test` brings in 
version of `test(...)` below that is designed specifically for the bootcamp

> 这个教程需要引入一些不一样的包。 `import chisel3.tester.RawTester.test` 
将引入这个教程所特有的 `test(...)` 版本

---
# Basic Tester implementation | 基础测试实现

Testers2 starts with the same basic operations as iotesters. Here's a brief summary of the basic
functionality mapping between the older iotesters and the new testers2

Testers2 最简单的操作跟 iotesters 中的相同。这里是一个两者功能对照的总结表。


|        | iotesters             | testers2            |
| :----  | :---                  | :---                |
| poke   | poke(c.io.in1, 6)     | c.io.in1.poke(6.U)    |
| peek   | peek(c.io.out1)       | c.io.out1.peek(6.U)   |
| expect | expect(c.io.out1, 6)  | c.io.out1.expect(6.U) |
| step   | step(1)               | c.io.clock.step(1)  |
| initiate | Driver.execute(...) { c => | test(...) { c => |


Let's start by looking at the simple pass through module from 2.1

让我们从 2.1 小节中最简单的 pass through 模块开始

In [ ]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

Using the old style a simple test would look like this

如果使用老版本的测试可能是下面这个样子的：

```scala
val testResult = Driver(() => new Passthrough()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // Set our input to value 0
    expect(c.io.out, 0)  // Assert that the output correctly has 0
    poke(c.io.in, 1)     // Set our input to value 1
    expect(c.io.out, 1)  // Assert that the output correctly has 1
    poke(c.io.in, 2)     // Set our input to value 2
    expect(c.io.out, 2)  // Assert that the output correctly has 2
  }
}
assert(testResult)   // Scala 代码: 如果 testResult == false, 将会报错
println("成功!!") // Scala 代码，如果我们运行到了这里，那么我们的验证就通过啦！
```



In [ ]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

>Just to illustrate the way testers2 advances the clock we can
add some `step` operations to the previous examples.

> 在 testers2 中，我们可以使得时钟前进。比如在前面的例子中，我们可以加入一些 `step` 操作。

In [ ]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.clock.step(1)    // 使得时钟快进
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.clock.step(1)    // 使得时钟快进
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.clock.step(1)    // 使得时钟快进
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

---
## What to notice in the above example | 在上面的例子中要注意什么

Testers2 test method requires a bit less boiler plate. What was the `PeekPokeTester` is now
built into the process.

The `poke` and `expect` methods are now part of each individual `io` element.
This gives important hints the the tester to make better checking of types.
The `peek` and `step` operations are also now methods on `io` elements.

Another difference is that values poked and expected are Chisel literals.
Although pretty simple here, it also provides stronger checking in more advanced and interesting examples.
This will be further enhanced with coming improvements in the ability to specify `Bundle` literals


Testers2 测试方法没有那么的死板。之前的 `PeekPokeTester` 现在已经在进展中。

现在 `poke` 和 `expect` 方法已经是每一个 `io` 元素的一部分了。
这给了测试者检查变量类型一个非常重要的线索。
`peek` 和 `step` 操作现在也是每一个 `io` 元素的一部分了。
 
另外一个不同点在于变量 `poke` `peek` 的都是 Chisel 文本类型变量。
尽管在这里不需要这么复杂，但是在一些更加高级和有趣的测试中，这也提供了一个很重要的检查依据。
在未来的优化中，一些特殊 `Bundle` 文本类型的变量将会有进一步的提升。


# Modules with Decoupled Interfaces | 带有 Decoupled 端口
In this section we will look at some of the tester2's tools for working with `Decoupled` interfaces.
`Decoupled` takes a chisel data type and provides it with `ready` and `valid` signals.
Testers2 provides some nice tools for automating and reliably testing these interfaces.

在本小节中，我们将会看一些 testers2 对 `Decoupled` 端口测试的例子。
`Decoupled` 需要一个 chisel 数据类型，并且提供 `ready` 和 `valid` 信号。
Testers2 为自动化地、可靠地测试这些端口提供了一些很棒的工具。


## A queue example | 一个队列例子
The `QueueModule` passes through data whose type is determined by `ioType`. There are `entries` state elements inside the `QueueModule` meaning it can hold that many elements before it exerts backpressure.

`QueueModule` 模块传递类型为 `ioType`的数据。在 `QueueModule` 中有 `entries` 个排队的元素，这意味着在它在产生背压时能够保持这么多个元素。

In [ ]:
case class QueueModule[T <: Data](ioType: T, entries: Int) extends MultiIOModule {
  val in = IO(Flipped(Decoupled(ioType)))
  val out = IO(Decoupled(ioType))
  out <> Queue(in, entries)
}


> Note the `case` class modifer is not generally required but seems to be in order for
this example to be re-used in multiple cells in Jupyter

> 注意，通常 案例类 并不是需要的，但是为了在 Jupyer 中可以被多个代码块使用，我们在这个例子中就需要加上它。

## EnqueueNow and expectDequeueNow
*testers2* has some built in methods for dealing with circuits with Decoupled interfaces in the IOs. In this example we will see how to insert and extract values from the `queue`. 

*testers2* 有一些自带的处理 Decoupled 端口的方法。在这个例子中，我们将会看到怎么样从队列中输入和提取值。

| method | description |
| :---   | :---        |
| enqueueNow | Add (enqueue) one element to a `Decoupled` input interface |
| expectDequeueNow | Removes (dequeues) one element from a `Decoupled` output interface |
---


>Note: There is some required boiler plate `initSource`, `setSourceClock`, etc that is necessary to ensure that the `ready` and `valid` fields are
all initialized correctly at the beginning of the test.

> 注意：我们有一些死板的参数需要设置，比如`initSource`, `setSourceClock`等。这是为了确保`ready` 和 `valid` 信号在测试开始的时候被正确地初始化了。


In [ ]:
test(QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(200){ i => i.U }

    testVector.zip(testVector).foreach { case (in, out) =>
      c.in.enqueueNow(in)
      c.out.expectDequeueNow(out)
    }
}

## EnqueueSeq and DequeueSeq 
Now we are going to introduce two new methods that deal with enqueuing and dequeuing operations in single operations.

现在我们要介绍两个新的方法，它们能够通过一次操作进行连续的入队、出队。

| method | description |
| :---   | :---        |
| enqueueSeq | Continues to add (enqueue) elements from the `Seq` to a `Decoupled` input interface, one at a time, until the sequence is exhausted |
| expectDequeueSeq | Removes (dequeues) elements from a `Decoupled` output interface, one at a time, and compares each one to the next element of the `Seq` |
---

> Note: The example below works fine but, as written, the `enqueueSeq` must finish before the `expectDequeueSeq` can begin. This example would fail if the `testVector`'s size is made larger than the queue depth, because the queue would fill up and not be able to complete the `enqueueSeq`. Try it yourself to see what the failure looks like. In the next section we will show to construct this type of test properly.

> 注意：下面的这个例子虽然能够成功运行但是，在 `expectDequeueSeq` 开始前， `enqueueSeq` 必须结束。如果 `testVector` 比队列的深度更大，那么这个例子将会失败。因为队列将会被填满，所以不能够完成 `enqueueSeq`。你可以自己去尝试错误的时候会发生什么。在下一个小节中我们将会展示怎么样合适地构造这种类型的测试。

In [ ]:
test(QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(100){ i => i.U }

    c.in.enqueueSeq(testVector)
    c.out.expectDequeueSeq(testVector)
}

> One more important takeaway from the last section is that the functions we just saw, `enqueueNow`, 
`enqueueSeq`, `expectDequeueNow`, and `expectDequeueSeq` are not complicated special case logic in testers2.
Rather they are examples of the kinds of harness building that testers2 encourages you to build from the testers2 primitives. To see how these methods are implemented check out [TestAdapters.scala](https://github.com/ucb-bar/chisel-testers2/blob/d199c5908828d0be5245f55fce8a872b2afb314e/src/main/scala/chisel3/tester/TestAdapters.scala)

> 在最后一个小节中，另外一个重要的感想是，我们刚刚看到的这些函数，`enqueueNow`, 
`enqueueSeq`, `expectDequeueNow`, 和 `expectDequeueSeq` ，都不是在 testers2 里面复杂的、特殊的实例逻辑。
但它们却是 testers2 鼓励你从基本操作里面构建新的测试逻辑的例子。从 [TestAdapters.scala](https://github.com/ucb-bar/chisel-testers2/blob/d199c5908828d0be5245f55fce8a872b2afb314e/src/main/scala/chisel3/tester/TestAdapters.scala) 可以看到它们实现的逻辑。

# Fork and Join in testers2

In this section we will look at running sections of a unit test concurrently. In order to do this we will introduce two new features of testers2.

在本小节中，我们将会看到同时测试一个模块的例子。为了实现它，我们要介绍两个 testers2 里面的新功能。

| method | description |
| :---   | :---        |
| fork   | launches a concurrent code block, additional forks can be run concurrently to this one via the .fork appended to end of the code block of the preceeding fork |
| join | re-unites multiple related forks back into the calling thread |
---

In the example below two `fork`s are chained together, and then `join`ed. In the first `fork` block the `enqueueSeq` will continue to add elements until exhausted. The second `fork` block will `expectDequeueSeq` on each cycle when data is available.

>The threads created by fork are run in a deterministic order, largely according to their order as specified in code, and certain bug-prone operations that depend on other threads are forbidden with runtime checks. 

在下面的例子中，两个 `fork` 被联系在一起，然后被 `join`。第一个 `fork` 模块里 `enqueueSeq` 将会持续地向队列里输入一个元素，直到结束。第二个`fork` 模块里，当数据可用时，每一个时钟周期都将会 `expectDequeueSeq`。


In [ ]:

test(QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(300){ i => i.U }

    fork {
        c.in.enqueueSeq(testVector)
    }.fork {
        c.out.expectDequeueSeq(testVector)
    }.join()
}

## Using Fork and Join with GCD
In this section we will use the fork join methods to implement tests of *Greatest Common Denominator* **GCD**.
Let's start by defining our IO bundles. We are going to add a bit of boiler plate here to allow us to use `Bundle` *literals*. As the comments say, it is hoped that we will soon have support for autogeneration of the literal support code.

在本小节中，我们将会使用 fork 和 join 方法来实现*最大公因数* **GCD** 的测试。
让我们从定义我们的 IO 开始。我们将会在这里添加一些刻板的东西，使得我们可以使用`Bundle` *文本变量*。
正如注释中讲的那样，我们希望可以早些得到支持`Bundle` *文本变量*类型的代码。也即是`Decoupled(new GcdOutputBundle(width))`。

In [ ]:
class GcdInputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
}

In [ ]:
class GcdOutputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
  val gcd    = UInt(w.W)
}

Now let's look at a *Decoupled* version of **GCD**. We've modified it a bit here to use the `Decoupled` wrapper that adds a `ready` and a `valid` signal to the input and output Bundle. The `Flipped` wrapper takes the `Decoupled` `GcdInputBundle` which by default is created as an output and converts each field to the opposite direction (recursively). The data elements of the bundled arguments to `Decoupled` are placed in the top level field `bits`. 

现在我们来看看一个 *Decoupled* 版本的 **GCD**。在这里我们已经修改了一点点以使用 `Decoupled` 封装器，这可以给输入和输出端口添加一个`ready` 和一个 `valid` 信号。 `Flipped` 封装器包裹 `Decoupled` `GcdInputBundle`，因为它默认是一个输出；使用这个封装器后它的方向就反过来了（递归地）。在 bundle 里面的参数被 `Decoupled` 调整到顶部 `bits` 变量里了。

In [ ]:
/**
  * Compute GCD using subtraction method.
  * Subtracts the smaller of registers x and y from the larger until register y is zero.
  * value input register x is then the Gcd
  * returns a packet of information with the two input values and their GCD
  */
class DecoupledGcd(width: Int) extends MultiIOModule {

  val input = IO(Flipped(Decoupled(new GcdInputBundle(width))))
  val output = IO(Decoupled(new GcdOutputBundle(width)))

  val xInitial    = Reg(UInt())
  val yInitial    = Reg(UInt())
  val x           = Reg(UInt())
  val y           = Reg(UInt())
  val busy        = RegInit(false.B)
  val resultValid = RegInit(false.B)

  input.ready := ! busy
  output.valid := resultValid
  output.bits := DontCare

  when(busy)  {
    // during computation keep subtracting the smaller from the larger
    when(x > y) {
      x := x - y
    }.otherwise {
      y := y - x
    }
    when(y === 0.U) {
      // when y becomes zero computation is over, signal valid data to output
      output.bits.gcd := x
      output.bits.value1 := xInitial
      output.bits.value2 := yInitial
      output.bits.gcd := x
      output.valid := true.B
      busy := false.B
    }
  }.otherwise {
    when(input.valid) {
      // valid data available and no computation in progress, grab new values and start
      val bundle = input.deq()
      x := bundle.value1
      y := bundle.value2
      xInitial := bundle.value1
      yInitial := bundle.value2
      busy := true.B
      resultValid := false.B
    }
  }
}


Our test looks pretty much the same as the earlier Queue tests.
But there's more going on because the computation take multiple cycles so the input enqueue process is blocked as each GCD is computed.
The good news is that test side of this is simple and consistent across different Decoupled circuits.

Also introduced here is the new Chisel3 `Bundle` literal notation. Consider the line

```scala
new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
```
`GcdInputBundle` defined above has two fields `value1` and `value2`.
We create a bundle literal by first creating a bundle then calling its `.Lit` method.
That method takes a variable argument list of key/value pairs, where the key (e.g. `_.value`) is the field name and the value (e.g. x.U) is a chisel hardware literal, the Scala `Int` x is converted into a Chisel `UInt` literal.
The `_.` in front of the field name is necessary to bind the name value to the bundle internals. 

>This may not be the perfect notation but in extensive development discussions it was viewed as
the best balance between minimizing boilerplate and the notational limitations available in Scala.


我们的测试看起来就像之前队列测试一样棒。
但是有更多需要改进的地方，因为计算要花数个周期，所以当 GCD 在计算的时候，输入是被无视的。
好消息是这个的测试端是比较简单的，并且在不同的端口都是恒定不变的。

在这里也介绍一下新的 Chisel3 `Bundle` 文字记号。思考下面这行代码：

```scala
new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
```
在上面定义的`GcdInputBundle`中，有两个文本变量 `value1` and `value2`。
我们首先创建一个 bundle，然后调用`.Lit`方法，创造了一个 `Bundle` 文本变量。
这个方法需要一个键/值对变量列表，键（比如说`_.value`）是应用的变量名，值（比如说 x.U）是 chisel 硬件文本变量，Scala `Int` x 将会被转变为一个 chisel 的 `UInt` 文本变量。
在每一个应用的变量名前面的`_.`用来绑定变量名以及 bundle 内部。

> 这可能并不是一个很棒的记号，但是在扩展发展讨论中，这被看作是最小化刻板语句以及 Scala 记号使用限制之间最好的平衡。

In [ ]:
test(new DecoupledGcd(16)) { dut =>
  dut.input.initSource().setSourceClock(dut.clock)
  dut.output.initSink().setSinkClock(dut.clock)

  val testValues = for { x <- 1 to 10; y <- 1 to 10} yield (x, y)
  val inputSeq = testValues.map { case (x, y) =>
    (new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
  }
  val resultSeq = testValues.map { case (x, y) =>
    new GcdOutputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U, _.gcd -> BigInt(x).gcd(BigInt(y)).U)
  }

  fork {
    dut.input.enqueueSeq(inputSeq)
  }.fork {
    dut.output.expectDequeueSeq(resultSeq)
  }.join()
}


---
# You're done! | 恭喜你，完成了本节内容的学习！

[Return to the top. | 回到顶层](#top)